# **Procesamiento de Lenguaje Natural**

## Maestría en Inteligencia Artificial Aplicada
#### Tecnológico de Monterrey
#### Prof Luis Eduardo Falcón Morales

### **Adtividad en Equipos Semanas 7 y 8 : LDA y LMM audio-a-texto**

* **Nombres y matrículas:**

  * Fernando Omar Salazar Ortiz - A01796214
  * Carlos Aaron Bocanegra Buitron - A01796345
  * Luis Enrique González González - A01795338
  * Gloria María Campos García - A01422345

* **Número de Equipo: 12**


* ##### **En cada ejercicio pueden importar los paquetes o librerías que requieran.**

* ##### **En cada ejercicio pueden incluir las celdas y líneas de código que deseen.**

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
#!pip install scipy==1.13.1

In [5]:
!pip install gensim --upgrade --user

In [6]:
#Para obtener las versiones disponibles
#!pip index versions numpy

In [7]:
#!pip install numpy==1.26.4

In [8]:
#!python -m spacy download es_core_news_lg

In [9]:
#imports
import torch
from transformers import pipeline
from transformers import AutoProcessor, AutoModelForSpeechSeq2Seq
import librosa   # paquete para audio y música
from IPython.display import Audio
import pickle
import re

import gensim
import gensim.corpora as corpora

import nltk
from nltk.corpus import stopwords
nltk.download('punkt_tab')
nltk.download('stopwords')

import spacy #Para aplicar la lemmatization

from google.colab import userdata
from openai import OpenAI
import openai

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
#Se obtiene la api key de OpenAI que se usará para el LLM
api_key = userdata.get("OPENAI_API_KEY_PERSONAL")

if not api_key:
  raise ValueError("API key no encontrada en los secretos")

In [11]:
#nlp = spacy.load("es_core_news_lg")

# **Ejercicio 1:**

* #### **Liga de los audios de las fábulas de Esopo:** https://www.gutenberg.org/ebooks/21144

* #### **Descargar los 10 archivos de audio solicitados: 1, 4, 5, 6, 14, 22, 24, 25, 26, 27.**



In [12]:
nombres_audios = []
audios = []

**Se van a utilizar los archivos em formato mp3**

In [13]:
# Incluyan a continuación todas las celdas (de código o texto) que deseen...
video_urls = [
    "https://www.gutenberg.org/files/21144/mp3/21144-01.mp3",
    "https://www.gutenberg.org/files/21144/mp3/21144-04.mp3",
    "https://www.gutenberg.org/files/21144/mp3/21144-05.mp3",
    "https://www.gutenberg.org/files/21144/mp3/21144-06.mp3",
    "https://www.gutenberg.org/files/21144/mp3/21144-14.mp3",
    "https://www.gutenberg.org/files/21144/mp3/21144-22.mp3",
    "https://www.gutenberg.org/files/21144/mp3/21144-24.mp3",
    "https://www.gutenberg.org/files/21144/mp3/21144-25.mp3",
    "https://www.gutenberg.org/files/21144/mp3/21144-26.mp3",
    "https://www.gutenberg.org/files/21144/mp3/21144-27.mp3"
]

for url in video_urls:
    !wget "{url}"
    nombre_archivo = url.split("/")[-1]
    nombres_audios.append(nombre_archivo)

--2025-06-06 16:54:17--  https://www.gutenberg.org/files/21144/mp3/21144-01.mp3
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1163470 (1.1M) [audio/mpeg]
Saving to: ‘21144-01.mp3’

21144-01.mp3        100%[===================>]   1.11M  2.93MB/s    in 0.4s    

2025-06-06 16:54:17 (2.93 MB/s) - ‘21144-01.mp3’ saved [1163470/1163470]

--2025-06-06 16:54:17--  https://www.gutenberg.org/files/21144/mp3/21144-04.mp3
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1077749 (1.0M) [audio/mpeg]
Saving to: ‘21144-04.mp3’

21144-04.mp3        100%[===================>]   1.03M  2.37MB/s    in 0.4s    

2025-06-06 16:54:1

In [14]:
nombres_audios

['21144-01.mp3',
 '21144-04.mp3',
 '21144-05.mp3',
 '21144-06.mp3',
 '21144-14.mp3',
 '21144-22.mp3',
 '21144-24.mp3',
 '21144-25.mp3',
 '21144-26.mp3',
 '21144-27.mp3']

In [15]:
# Se obtine la informacion de los audios cargados
for i in range(10):
  audio_path = nombres_audios[i]
  audio, sample_rate = librosa.load(audio_path)
  audios.append(audio)
  print(f"Audio: {nombres_audios[i]}, Duración del audio: {len(audio)/sample_rate:.1f} segundos")

Audio: 21144-01.mp3, Duración del audio: 72.6 segundos
Audio: 21144-04.mp3, Duración del audio: 67.1 segundos
Audio: 21144-05.mp3, Duración del audio: 58.4 segundos
Audio: 21144-06.mp3, Duración del audio: 71.8 segundos
Audio: 21144-14.mp3, Duración del audio: 58.1 segundos
Audio: 21144-22.mp3, Duración del audio: 57.2 segundos
Audio: 21144-24.mp3, Duración del audio: 64.5 segundos
Audio: 21144-25.mp3, Duración del audio: 56.5 segundos
Audio: 21144-26.mp3, Duración del audio: 63.7 segundos
Audio: 21144-27.mp3, Duración del audio: 60.3 segundos


# **Ejercicio 2a:**

* #### **Comenten el por qué del modelo seleccionado para extracción del texto de los audios.**

* #### **Extraer el contenido de los audios en texto.**

* #### **Sugerencia:** pueden extraerlo en un formato de diccionario, clave:valor $→$ {audio01:fabula01, ...}

**Decidimos intentar los modelos de HuggingFace, OpenAI y Google para los audios 1 y 6 y tras una revisión manual a mi parecer el modelo de  whisper de Open AI con Hugging Face fue mejor, a continuación de muestra la comparación de las transcripciones generdas.**


**Audio 1**

In [16]:
Audio('21144-01.mp3')

whisper de Open AI con Hugging Face

 Las Fábulas de <mark>Sopo</mark> Grabado para LibriVox.org por Paulino www.paulino.info Fábula número 61 El Lobo y el Cordero en el Templo Dándose cuenta de que era perseguido por un lobo Un pequeño corderito decidió refugiarse en un templo cercano Lo llamó lobo y le dijo que si el sacrificador lo encontraba allí adentro Lo <mark>inmolaría</mark> a su dios Mejor así, replicó el cordero Prefiero ser víctima para un dios A tener que perecer en tus colmillos Si sin remedio vamos a ser sacrificados Más nos vale que sea con el mayor honor Fin de la fábula Esta es una grabación del dominio público

whisper-1 de Open AI

Las fábulas de <mark>Sopo</mark> Grabado para LibriVox.org por Paulino www.paulino.info Fábula número 61 El lobo y el cordero en el templo Dándose cuenta de que era perseguido por un lobo, un pequeño corderito decidió refugiarse en un templo cercano. Lo llamó lobo y le dijo que si el sacrificador lo encontraba allí adentro, lo <mark>enmolaría</mark> a su dios. Mejor así, replicó el cordero, prefiero ser víctima para un dios a tener que perecer en tus colmillos. Si sin remedio vamos a ser sacrificados, más nos vale que sea con el mayor honor. Fin de la fábula Esta es una grabación del dominio público.

Google

las fábulas de <mark>Esopo</mark> grabado para librivox.org por Paulino
 Dándose cuenta de que era perseguido por un lobo un pequeño corderito decidió refugiarse en un templo cercano lo llamó el lobo y le dijo que si el sacrificador lo encontraba allí adentro lo <mark>emular</mark>, su Dios mejor así replicó el cordero Prefiero ser víctima para un Dios a tener que perecer en tus colmillos sí sin remedio vamos a ser sacrificados más nos vale que sea con el mayor honor fin de la fábula Esta es una grabación del dominio público.

**Audio 6**

In [17]:
Audio('21144-06.mp3')

whisper de Open AI con Hugging Face

 Las fábulas de Esopo, grabado para LibriVox.org por Alejandro González Calderón. Fábula número 66, El Lobo y el Asno. Un lobo fue elegido rey entre sus congéneres y decretó una ley ordenando que lo que cada uno <mark>capturase</mark> en la casa lo pusiera en común y lo repartiese por partes iguales entre todos. De esta manera ya no tendrían los lobos que devorarse unos a otros en épocas de hambre. Pero en eso le escuchó un asno que estaba por ahí cerca y moviendo sus orejas le dijo Magnífica idea ha brotado de tu corazón Pero, ¿por qué has escondido todo tu botín en tu cueva <mark>Llevándolo</mark> a la comunidad y <mark>repártelo</mark> también como lo has decretado? El lobo descubierto y confundido derogó su ley Si alguna vez llegas a tener poder de legislar Sé el primero en cumplir tus propias leyes Fin de la fábula Esta grabación es de dominio público

whisper-1 de Open AI

Las fábulas de Esopo, grabado para LibriVox.org por Alejandro González Calderón. Fábula número 66. El Lobo y el Asno. Un lobo fue elegido rey entre sus congéneres y decretó una ley ordenando que lo que cada uno <mark>capturase</mark> en la casa, lo pusiera en común y lo repartiese por partes iguales entre todos. De esta manera ya no tendrían los lobos que devorarse unos a otros en épocas de hambre. Pero en eso le escuchó un asno, que estaba por ahí cerca, y moviendo sus orejas le dijo ¡Magnífica idea ha brotado de tu corazón! Pero, ¿por qué has escondido todo tu botín en tu cueva? <mark>Llévalo</mark> a la comunidad y <mark>repártelo</mark> también como lo has decretado. El lobo, descubierto y confundido, derogó su ley. Si alguna vez llegas a tener poder de legislar, sé el primero en cumplir tus propias leyes. Fin de la fábula. Esta grabación es de dominio público.

Google

Las fábulas de Esopo grabado para librivox.org por Alejandro González Calderón fábula número 66 El lobo y el asno un lobo fue elegido Rey entre sus congéneres y decretó una ley ordenando que lo que cada uno <mark>captura en</mark> la casa lo pusieran en común y lo repartiese por partes iguales entre todos de esta manera ya no tendría Los Lobos que devorarse unos a otros en épocas de hambre pero en eso le escuchó un asno que estaba por ahí cerca y moviendo sus orejas le dijo magnífica idea ha brotado de tu corazón Pero por qué has escondido todo tu botín en tu cueva <mark>llevando</mark> a la comunidad y <mark>reparte lo</mark> también como lo has decretado el lobo descubierto y confundido derogó su ley.
 Si alguna vez llegas a tener poder de legislar, sé el primero en cumplir tus propias leyes fin de la fábula esta grabación es de dominio público.

In [18]:
# Incluyan a continuación todas las celdas (de código o texto) que deseen...

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Usando dispositivo: {device}")

Usando dispositivo: cuda


In [19]:
model_id = "openai/whisper-large-v3-turbo"    # una versión ligera de v3
processor = AutoProcessor.from_pretrained(model_id)
modelo = AutoModelForSpeechSeq2Seq.from_pretrained(model_id).to(device)

preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.71M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

...

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.26k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.62G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.77k [00:00<?, ?B/s]

In [20]:
# Inicializamos el pipeline:
pipe = pipeline("automatic-speech-recognition",
                model=modelo,
                tokenizer=processor.tokenizer,
                feature_extractor=processor.feature_extractor,
                torch_dtype=torch.float32,      #torch.float16 if torch.cuda.is_available() else torch.float32,
                device=device
                )

Device set to use cuda


In [21]:
textos = {}

In [22]:
# Extraemos los textos de los audios
i = 0
for audio in audios:
  result = pipe(audio,
                return_timestamps=True,   # Para audios mayores a 30 segs.
                generate_kwargs={"language":"es"}  # Lo detecta automático, pero se le puede indicar el idioma.
                )
  textos[nombres_audios[i]] = result['text']
  i += 1

/usr/local/lib/python3.11/dist-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
You have passed language=es, but also have set `forced_decoder_ids` to [[1, None], [2, 50360]] which creates a conflict. `forced_decoder_ids` will be ignored in favor of language=es.
Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.
Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.
Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.


In [23]:
textos

{'21144-01.mp3': ' Las Fábulas de Sopo Grabado para LibriVox.org por Paulino www.paulino.info Fábula número 61 El Lobo y el Cordero en el Templo Dándose cuenta de que era perseguido por un lobo Un pequeño corderito decidió refugiarse en un templo cercano Lo llamó lobo y le dijo que si el sacrificador lo encontraba allí adentro Lo inmolaría a su dios Mejor así, replicó el cordero Prefiero ser víctima para un dios A tener que perecer en tus colmillos Si sin remedio vamos a ser sacrificados Más nos vale que sea con el mayor honor Fin de la fábula Esta es una grabación del dominio público',
 '21144-04.mp3': ' Las Fábulas de Esopo, grabado para LibriVox.org por Roberto Antonio Muñoz Fábula Número 64 El Lobo y la Coruja A un lobo que comía un hueso, se le atragantó el hueso en la garganta y corría por todas partes en busca de auxilio. Encontró en su correr a una grulla y le pidió que le salvara de aquella situación, y que enseguida le pagaría por ello. Aceptó la grulla e introdujo su cabeza 

In [24]:
# Guardar en archivo .pkl
'''
with open("/content/drive/MyDrive/Colab Notebooks/NLP/Actividades/ActividadSemana7y8_LDA_LLM/textos.pkl", "wb") as archivo:
    pickle.dump(textos, archivo)
'''

'\nwith open("/content/drive/MyDrive/Colab Notebooks/NLP/Actividades/ActividadSemana7y8_LDA_LLM/textos.pkl", "wb") as archivo:\n    pickle.dump(textos, archivo)\n'

# **Ejercicio 2b:**

* #### **Eliminar el inicio y final comunes de los textos extraídos de cada fábula.**

* #### **Sugerencia:** Pueden guardar esta información en un archivo tipo JSON, para que al estar probando diferentes opciones en los ejercicios siguientes, puedan recuperar rápidamente la información de cada video/fábula.

In [25]:
#Se leen las fábulas sin limpiar
with open('/content/drive/MyDrive/Colab Notebooks/NLP/Actividades/ActividadSemana7y8_LDA_LLM/textos.pkl', 'rb') as f:
    textos = pickle.load(f)

In [26]:
# Incluyan a continuación todas las celdas (de código o texto) que deseen...
textos_limpios = {}

In [27]:
#Se limpian los textos para que no incluyan ni el inicio ni final que no es parte de la fábula
for clave, valor in textos.items():
  partes = re.split(r'[0-9]{2}[,\.]? ', valor)
  #print(partes[1])
  fabula_limpia = re.split(r'(Fin de|fin de)', partes[1])
  #print(fabula_limpia[0])
  textos_limpios[clave] = fabula_limpia[0]
textos_limpios

{'21144-01.mp3': 'El Lobo y el Cordero en el Templo Dándose cuenta de que era perseguido por un lobo, un pequeño corderito decidió refugiarse en un templo cercano. Lo llamó lobo y le dijo que si el sacrificador lo encontraba allí adentro, lo inmolaría a su dios. Ahora sí, replicó el cordero, prefiero ser víctima para un dios a tener que perecer en tus colmillos. Si sin remedio vamos a ser sacrificados, más nos vale que sea con el mayor honor. ',
 '21144-04.mp3': 'El Lobo y la Coruja A un lobo que comía un hueso, se le atragantó el hueso en la garganta y corría por todas partes en busca de auxilio. Encontró en su correr a una grulla y le pidió que le salvara de aquella situación, y que enseguida le pagaría por ello. Aceptó la grulla e introdujo su cabeza en la boca del lobo, sacando de la garganta el hueso atravesado. Pidió entonces la cancelación de la paga convenida. Oye amiga, dijo el lobo, ¿no crees que es suficiente paga con haber sacado tu cabeza sana y salva de mi boca? Nunca hag

In [28]:
# Guardar en archivo .pkl las fábulas limpias
'''
with open("/content/drive/MyDrive/Colab Notebooks/NLP/Actividades/ActividadSemana7y8_LDA_LLM/textos_limpios.pkl", "wb") as archivo:
    pickle.dump(textos_limpios, archivo)
'''

'\nwith open("/content/drive/MyDrive/Colab Notebooks/NLP/Actividades/ActividadSemana7y8_LDA_LLM/textos_limpios.pkl", "wb") as archivo:\n    pickle.dump(textos_limpios, archivo)\n'

In [29]:
#Se leen las fábulas
with open('/content/drive/MyDrive/Colab Notebooks/NLP/Actividades/ActividadSemana7y8_LDA_LLM/textos_limpios.pkl', 'rb') as f:
    textos_limpios = pickle.load(f)

In [30]:
textos_limpios

{'21144-01.mp3': 'El Lobo y el Cordero en el Templo Dándose cuenta de que era perseguido por un lobo, un pequeño corderito decidió refugiarse en un templo cercano. Lo llamó lobo y le dijo que si el sacrificador lo encontraba allí adentro, lo inmolaría a su dios. Ahora sí, replicó el cordero, prefiero ser víctima para un dios a tener que perecer en tus colmillos. Si sin remedio vamos a ser sacrificados, más nos vale que sea con el mayor honor. ',
 '21144-04.mp3': 'El Lobo y la Coruja A un lobo que comía un hueso, se le atragantó el hueso en la garganta y corría por todas partes en busca de auxilio. Encontró en su correr a una grulla y le pidió que le salvara de aquella situación, y que enseguida le pagaría por ello. Aceptó la grulla e introdujo su cabeza en la boca del lobo, sacando de la garganta el hueso atravesado. Pidió entonces la cancelación de la paga convenida. Oye amiga, dijo el lobo, ¿no crees que es suficiente paga con haber sacado tu cabeza sana y salva de mi boca? Nunca hag

# **Ejercicio 3:**

* #### **Apliquen el proceso de limpieza que consideren adecuado.**

* #### **Justifiquen los pasos de limpieza utilizados. Tomen en cuenta que el texto extraído de cada fábula es relativamente pequeño.**

* #### **En caso de que decidan no aplicar esta etapa de limpieza, deberán justificarlo.**

**Se va a aplicar el siguiente preprocesamiento**
1. Se va a convertir todo el texto a minúsculas
2. Se van a omitir los caracteres no alfabeticos para eliminiar los signos de puntuación y que no sean considerados como tokens independientes.
2. Se quitaran las stopwords para evitar que sean consideradas dentro de las 20 palabras que van a definir a cada tópico.

No se aplicaron mas procesos de limpieza porque los textos son pequeños y no se querían perder más tokens.

In [31]:
#stopwords.words('spanish')

In [32]:
# Incluyan a continuación todas las celdas (de código o texto) que deseen...
#nlp = spacy.load("es_core_news_lg")

def clean_doc(doc):

  #Convertir a minusculas
  docTmp = doc.lower()

  #Reemplazar todos los caracteres no alfabeticos por espacios en blanco
  docTmp2 = re.sub(r'[^a-záéíóúñ]', ' ', docTmp)

  #Tokenizacion usando word_tokenize
  tokensTmp = nltk.word_tokenize(docTmp2)

  #Remover stopwords
  tokensTmp2 = [token for token in tokensTmp if token not in stopwords.words('spanish')]

  #Aplicar Lemmatization

  #tokensTmp3 = [nlp(token)[0].lemma_ for token in tokensTmp2]

  tokens = tokensTmp2

  return tokens


textos_preprocesados = [clean_doc(x) for x in textos_limpios.values()]
textos_preprocesados[0]


['lobo',
 'cordero',
 'templo',
 'dándose',
 'cuenta',
 'perseguido',
 'lobo',
 'pequeño',
 'corderito',
 'decidió',
 'refugiarse',
 'templo',
 'cercano',
 'llamó',
 'lobo',
 'dijo',
 'si',
 'sacrificador',
 'encontraba',
 'allí',
 'adentro',
 'inmolaría',
 'dios',
 'ahora',
 'replicó',
 'cordero',
 'prefiero',
 'ser',
 'víctima',
 'dios',
 'tener',
 'perecer',
 'colmillos',
 'si',
 'remedio',
 'vamos',
 'ser',
 'sacrificados',
 'vale',
 'mayor',
 'honor']

# **Ejercicio 4:**

In [33]:
# Incluyan a continuación todas las celdas (de código o texto) que deseen...

In [34]:
id2word = corpora.Dictionary(textos_preprocesados)

for id in range(5):
    print(id2word[id])

adentro
ahora
allí
cercano
colmillos


In [35]:
# Generamos el corpus con las tuplas (índice, frecuencia):
corpus = []
for i in range(len(textos_preprocesados)):
  corpus.append(id2word.doc2bow(textos_preprocesados[i]))
print(corpus[0])

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 2), (7, 1), (8, 1), (9, 1), (10, 2), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 3), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 2), (28, 2), (29, 2), (30, 1), (31, 1), (32, 1), (33, 1)]


In [36]:
lda_model =[]
for i in range(len(corpus)):
  lda_model.append(gensim.models.ldamodel.LdaModel(corpus=[corpus[i]],
                                              id2word=id2word,
                                              num_topics=1,        # total de tópicos que deseas buscar...
                                              chunksize=10,        # número de documentos a analizar en cada chunk de entrenamiento
                                              passes=10,           # número de pasadas al corpus durante el entrenamiento.
                                              alpha='auto',        # alfa: similaridad document-topics
                                              eta='auto',          # beta: similaridad topic-words
                                              random_state=42))


In [37]:
for i in range(len(corpus)):
  print(lda_model[i].print_topics(num_topics=1, num_words=20))

[(0, '0.011*"lobo" + 0.008*"templo" + 0.008*"si" + 0.008*"ser" + 0.008*"cordero" + 0.008*"dios" + 0.005*"sacrificador" + 0.005*"cuenta" + 0.005*"corderito" + 0.005*"colmillos" + 0.005*"decidió" + 0.005*"dándose" + 0.005*"perseguido" + 0.005*"adentro" + 0.005*"víctima" + 0.005*"remedio" + 0.005*"replicó" + 0.005*"tener" + 0.005*"encontraba" + 0.005*"cercano"')]
[(0, '0.013*"lobo" + 0.010*"hueso" + 0.010*"paga" + 0.008*"garganta" + 0.008*"cabeza" + 0.008*"pidió" + 0.008*"boca" + 0.008*"grulla" + 0.005*"todas" + 0.005*"salva" + 0.005*"partes" + 0.005*"convenida" + 0.005*"favores" + 0.005*"aquella" + 0.005*"entonces" + 0.005*"ello" + 0.005*"aceptó" + 0.005*"amiga" + 0.005*"atravesado" + 0.005*"atragantó"')]
[(0, '0.010*"caballo" + 0.010*"cebada" + 0.008*"lobo" + 0.005*"complacer" + 0.005*"dejado" + 0.005*"llevó" + 0.005*"agradaba" + 0.005*"pasaba" + 0.005*"repuso" + 0.005*"parezca" + 0.005*"masticarla" + 0.005*"preferido" + 0.005*"actuar" + 0.005*"malvado" + 0.005*"sino" + 0.005*"campo" + 

In [38]:
lda_model[0].show_topics(num_topics=-1, num_words=20, formatted=False)

[(0,
  [('lobo', 0.010666588),
   ('templo', 0.007999926),
   ('si', 0.007999924),
   ('ser', 0.007999924),
   ('cordero', 0.007999924),
   ('dios', 0.007999924),
   ('sacrificador', 0.0053332695),
   ('cuenta', 0.0053332695),
   ('corderito', 0.0053332695),
   ('colmillos', 0.0053332695),
   ('decidió', 0.0053332695),
   ('dándose', 0.0053332695),
   ('perseguido', 0.0053332695),
   ('adentro', 0.0053332695),
   ('víctima', 0.0053332695),
   ('remedio', 0.0053332695),
   ('replicó', 0.0053332695),
   ('tener', 0.0053332695),
   ('encontraba', 0.0053332695),
   ('cercano', 0.005333269)])]

In [39]:
temas = {}

for i in range(10):
  _, topic = lda_model[i].show_topics(num_topics=-1, num_words=20, formatted=False)[0]
  palabras_clave = [palabra for palabra, _ in topic]
  temas[f"Fábula {i+1}"] = palabras_clave

In [40]:
temas['Fábula 10']

['león',
 'perro',
 'camino',
 'siquiera',
 'vio',
 'primero',
 'entres',
 'rugiendo',
 'retrocedió',
 'listo',
 'casa',
 'imprevistos',
 'atemorizado',
 'empresa',
 'siempre',
 'infeliz',
 'mismo',
 'encontró',
 'imaginabas',
 'mantente']

# **Ejercicio 5a y 5b:**

* #### **5a: Mediante el LLM que hayan seleccionado, generar un único enunciado que describa o resuma cada fábula.**

* #### **5b: Mediante el LLM que hayan seleccionado, generar tres posibles enunciados diferentes relacionados con la historia de la fábula.**

* #### **Sugerencia:** En realidad los dos incisos a y b se pueden obtener con un solo prompt que solicite la información y el formato correspondiente para cada una de estas partes. Por ejemplo, para cada fábula la salida puede ser un primer enunciado genérico que resume o describe dicha temática; seguido de tres enunciados, cada uno hablando sobre una situación o parte diferente de la fábula.

In [41]:
modelo = "gpt-4o-mini"
client = OpenAI(api_key = api_key)

In [42]:
primer_elemento = dict([next(iter(temas.items()))])

In [43]:
#primer_elemento

In [44]:
'''
temas = {
    "El zorro y las uvas": [
        "zorro", "hambre", "viñedo", "uvas", "altas", "intento", "fracaso", "orgullo",
        "engaño", "justificación", "insatisfacción", "deseo", "necesidad", "autoengaño",
        "aceptación", "dignidad", "madurez", "engaño propio", "moral", "cuentos"
    ]
}
'''

# Construimos el bloque de entrada
bloque = ""
for i, (titulo, palabras) in enumerate(temas.items(), start=1):
    bloque += f"""{i}. {titulo}
Palabras clave: {', '.join(palabras)}

"""

# Creamos el prompt general
prompt = f"""
A continuación te presento una lista de fábulas y sus respectivas palabras clave.
Sin embargo no proveo el título de la fábula, sino solamente un título genérico tal como 'Fábula 1'


Para cada fábula, realiza lo siguiente:

1. Escribe **un enunciado resumen** que capture el sentido de la historia.
2. Genera **tres enunciados adicionales** que se relacionen con la historia.

Formato de salida:
1. [Título de la fábula]
Resumen: ...
Relacionado 1: ...
Relacionado 2: ...
Relacionado 3: ...

{bloque}
"""

# Petición al modelo
response = client.chat.completions.create(
    model=modelo,
    messages=[{"role": "user", "content": prompt}]
)

# Mostrar resultado
print(response.choices[0].message.content)

1. Fábula 1  
Resumen: Un lobo engaña a un cordero diciendo que es un sacrificio a Dios, pero su verdadero objetivo es devorarlo.  
Relacionado 1: El cordero, cada vez que intentaba defenderse, solo escuchaba las mentiras del lobo.  
Relacionado 2: A pesar de estar en un templo, la maldad del lobo no tenía límites y buscaba una victima fácil.  
Relacionado 3: La astucia del lobo y la inocencia del cordero representan la lucha entre el bien y el mal.

2. Fábula 2  
Resumen: Un lobo atraganta un hueso mientras intenta recibir ayuda de una grulla, quien acepta ayudarlo a cambio de favores, pero termina siendo perjudicada.  
Relacionado 1: La grulla, al querer ayudar, se ve envuelta en una situación que no esperaba.  
Relacionado 2: El lobo, a pesar de ser tan dañino, muestra que a veces las amistades pueden resultar en problemas.  
Relacionado 3: Esta fábula ilustra que algunos favores pueden tener un alto costo, incluso si parecen pequeños.

3. Fábula 3  
Resumen: Un lobo, tras ser recha

# **Ejercicio 6:**

* #### **Incluyan sus conclusiones de la actividad audio-a-texto:**



1. Para trabajar con audios primero se debe de convertir a texto y después aplicar los procesamientos que hemos aprendido, sin embargo la conversión de audio a texto es un proceso que requiere alta cantidad de recursos. Por ello si no se utiliza una GPU la conversión a texto es mucho más tardada aún cuando solo se convirtieron 10 fábulas.
2. Cuando se aplica LDA a con un solo documento y un solo tópico, nos indica cuáles son las palabras más importantes en el documento. Y si se aplica a varios documentos y se buscan varios tópicos entonce cumple el objetivo de encontrar tópicos en común.
3. Si se especifica el formato de salida esperado con un ejemplo en el prompt, entonces se obtendrá el resultado esperado.

# **Fin de la actividad LDA y LMM: audio-a-texto**